<a href="https://colab.research.google.com/github/yumeniown/module_3-5/blob/main/ht_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Задания**

1. Подготовка данных:
    Загрузите датасет SMS сообщений, размеченных как спам или не спам. https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

2. Предобработка текста: Реализуйте лемматизацию или стемминг для текстов сообщений.
3. Векторизация текста: Примените мешок слов (BagofWords)и TF-IDF для векторизации текста.
4. Моделирование: Постройте модели для классификации сообщений как спам или не спам (модели на ваш выбор).
5. Оцените качество моделей с различными комбинациями предобработкии векторизации (сравните ROC-AUC,F1-score,accuracyит.д.).



**1. Подготовка данных: Загрузите датасет SMS сообщений,размеченных как спам или не спам**


In [34]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

# загрузка словарей
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
import kagglehub

path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/uciml/sms-spam-collection-dataset/versions/1


In [36]:
import os

print("Files in dataset directory:", os.listdir("/root/.cache/kagglehub/datasets/uciml/sms-spam-collection-dataset/versions/1"))


Files in dataset directory: ['spam.csv']


 Преобразуем метки и подготавливаем данные для спам или не спам сообщений

In [37]:
df = pd.read_csv("/root/.cache/kagglehub/datasets/uciml/sms-spam-collection-dataset/versions/1/spam.csv", encoding='ISO-8859-1')

df.columns = ['label', 'message', 'Unnamed_2', 'Unnamed_3', 'Unnamed_4']

df = df[['label', 'message']]

df['label'] = df['label'].map({'spam': 1, 'ham': 0})

In [38]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   int64 
 1   message  5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [40]:
df.describe()

,label
count,5572.000000
mean,0.134063
std,0.340751
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


**2. Предобработка текста: Реализуйте лемматизацию или стемминг для текстов сообщений**



In [41]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.upper()
    text = re.sub(r'\W', ' ', text) # удаляем пунктуацию и специальные символы
    words = [lemmatizer.lemmatize(word) for word in text.split()] # лемматизация
    return ' '.join(words)

df['processed_message'] = df['message'].apply(preprocess_text) # предобработка на каждое сообщение

# проверяем результат предобработки
df[['message', 'processed_message']].head()

,message,processed_message
0,"Go until jurong point, crazy.. Available only ...",GO UNTIL JURONG POINT CRAZY AVAILABLE ONLY IN ...
1,Ok lar... Joking wif u oni...,OK LAR JOKING WIF U ONI
2,Free entry in 2 a wkly comp to win FA Cup fina...,FREE ENTRY IN 2 A WKLY COMP TO WIN FA CUP FINA...
3,U dun say so early hor... U c already then say...,U DUN SAY SO EARLY HOR U C ALREADY THEN SAY
4,"Nah I don't think he goes to usf, he lives aro...",NAH I DON T THINK HE GOES TO USF HE LIVES AROU...


**3. Векторизация текста: Примените мешок слов (BagofWords)и TF-IDF для векторизации текста**

In [42]:
# применяем мешок слов Bag of Words для векторизации текста
bow_vectorizer = CountVectorizer(max_features=1000)  # Ограничим размер словаря для простоты
X_bow = bow_vectorizer.fit_transform(df['processed_message'])

# применяем TF-IDF для векторизации текста
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(df['processed_message'])

print("Размер матрицы Bag of Words:", X_bow.shape)
print(X_bow.toarray()[:5])
print()
print("Размер матрицы TF-IDF:", X_tfidf.shape)
print(X_tfidf.toarray()[:5])


Размер матрицы Bag of Words: (5572, 1000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Размер матрицы TF-IDF: (5572, 1000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Успешное преобразование текста в числовой вид

**4. Моделирование: Постройте модели для классификации сообщений как спам или не спам (модели на ваш выбор)**

In [44]:
# разделим данные на обучающую и тестовую выборки
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, df['label'], test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(X_tfidf, df['label'], test_size=0.2, random_state=42)

# логистическая регрессия Bag of Words
model_log_bow = LogisticRegression()
model_log_bow.fit(X_train_bow, y_train)
y_pred_bow = model_log_bow.predict(X_test_bow)

# логистическая регрессия TF-IDF
model_log_tfidf = LogisticRegression()
model_log_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_log_tfidf.predict(X_test_tfidf)

# случайный лес Bag of Words
model_rf_bow = RandomForestClassifier()
model_rf_bow.fit(X_train_bow, y_train)
y_pred_rf_bow = model_rf_bow.predict(X_test_bow)

# случайный лес TF-IDF
model_rf_tfidf = RandomForestClassifier()
model_rf_tfidf.fit(X_train_tfidf, y_train)
y_pred_rf_tfidf = model_rf_tfidf.predict(X_test_tfidf)

print("Логистическая регрессия Bag of Words:")
print("Accuracy:", accuracy_score(y_test, y_pred_bow))
print("Precision:", precision_score(y_test, y_pred_bow))
print("Recall:", recall_score(y_test, y_pred_bow))
print("F1 Score:", f1_score(y_test, y_pred_bow))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_bow))
print()
print("Логистическая регрессия TF-IDF:")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print("Precision:", precision_score(y_test, y_pred_tfidf))
print("Recall:", recall_score(y_test, y_pred_tfidf))
print("F1 Score:", f1_score(y_test, y_pred_tfidf))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_tfidf))
print()
print("Случайный лес Bag of Words:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf_bow))
print("Precision:", precision_score(y_test, y_pred_rf_bow))
print("Recall:", recall_score(y_test, y_pred_rf_bow))
print("F1 Score:", f1_score(y_test, y_pred_rf_bow))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_rf_bow))
print()
print("Случайный лес TF-IDF:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf_tfidf))
print("Precision:", precision_score(y_test, y_pred_rf_tfidf))
print("Recall:", recall_score(y_test, y_pred_rf_tfidf))
print("F1 Score:", f1_score(y_test, y_pred_rf_tfidf))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_rf_tfidf))

Логистическая регрессия Bag of Words:
Accuracy: 0.97847533632287
Precision: 0.9846153846153847
Recall: 0.8533333333333334
F1 Score: 0.9142857142857143
ROC-AUC: 0.9256303972366149

Логистическая регрессия TF-IDF:
Accuracy: 0.9721973094170404
Precision: 0.983739837398374
Recall: 0.8066666666666666
F1 Score: 0.8864468864468864
ROC-AUC: 0.9022970639032816

Случайный лес Bag of Words:
Accuracy: 0.979372197309417
Precision: 0.9847328244274809
Recall: 0.86
F1 Score: 0.9181494661921709
ROC-AUC: 0.9289637305699481

Случайный лес TF-IDF:
Accuracy: 0.979372197309417
Precision: 0.9847328244274809
Recall: 0.86
F1 Score: 0.9181494661921709
ROC-AUC: 0.9289637305699481


**5. Оцените качество моделей с различными комбинациями предобработкии векторизации (сравните ROC-AUC,F1-score,accuracyит.д.)**

Случайные леса с представлениями текста в ключе мешка слов Bag of Words и TF-IDF показали наилучшие результаты по точности и сбалансированности метрик (Precision, Recall и F1 Score), что делает их оптимальными для задачи классификации спама, так как они способны надежно отличать спам от не спама, минимизируя ошибки пропуска спама и ложные срабатывания. Также можно заметить, что логистическая регрессия BOW показыавет хорошие результаты в ROC-AUC, как и классификаций случайном лесом